# Writer Monad

## Outline

* Incentive for the Writer monad

* The Writer monad
  - Definition of the Writer monad
  
  - Helper functions
  
  - MonadWriter type class
  
  - Simple example

* Writer monad examples

In this lesson, we will learn about the Writer monad type and how you can use it. First we present the reason why introducing the Writer monad. Then we show the Writer monad definition and helper functions. We also look at the `MonadWriter` type class and in the end show some code examples that use the Writer monad.

## Incentive for Writer monad

In the previous lecture we learned that with the Reader monad we can more easily pass data around by accessing the environment variable of the Reader monad. 

But what if instead of reading that data we want to append data to our environment variable?

Let's look at an example program that asks the user for a password and perorms four checks:
- check1: Does the password have any duplicate letters (lower-upper case does not make a difference)

- check2: Is the password at least 10 characters long.

- check3: Are any symbols included in the password.

- check4: Is there at least one upper case and one lower case letter in the password.

In case one of the checks is not satisfied the program updates the password so that it passes the check.

If the update could influence a previous check the program performs it again.

The program keeps also track of performed checks. Once the recursion limit of 20 checks is passed the checks stop. 

After that the program asks the user to provide a new password to be checked again. 

In [ ]:
import Data.Char (toLower, toUpper)

type Password = String
type OperationsCount = Int
type CheckData = (OperationsCount, Password)

-- Removes duplicated characters
check1 :: CheckData -> CheckData
check1 (count,password) = if count > 20
    then (count + 1, "")
    else check2 (count + 1, filterPass password)
  where filterPass [] = []
        filterPass (x:xs) = x : filterPass (filter (filterFunc x) xs)
        filterFunc x = \char -> char `notElem` [toLower x,toUpper x]

-- Checks that the length of the password is not to small
check2 :: CheckData -> CheckData
check2 (count,password) = 
  if length password < 10
  then check1 (count + 1, extendedPassword)
  else check3 (count + 1, password)
  where shortage = 10 - length password
        passwordExtension = take shortage (drop count $ cycle allLetters)
        extendedPassword = password ++ passwordExtension

-- Checks if symbols are included
check3 :: CheckData -> CheckData
check3 (count,password) = if noSymbols
    then check4 (count + 1, extendedPassword)
    else check4 (count + 1, password)
  where noSymbols = all (`elem` allLetters) password
        passwordExtension = take 1 (drop count "@!#$%&()?=")
        extendedPassword = password ++ passwordExtension

-- Checks there are at least 1 upper and 1 smaller letters
check4 :: CheckData -> CheckData
check4 (count,password)  
  | allLower = check1 (count + 1, upperExtendedPassword)
  | allUpper = check1 (count + 1, lowerExtendedPassword)
  | otherwise = (count + 1, password)
  where allLower = password == map toLower password
        allUpper = password == map toUpper password
        upperExtendedPassword = password ++ take 1 (drop count $ cycle upperLetters)
        lowerExtendedPassword = password ++ take 1 (drop count $ cycle lowerLetters)

lowerLetters :: String
lowerLetters = "abcdefghijklmnopqrstuvwxyz"

upperLetters :: String
upperLetters = "ABCDEFGHIJKLMNOPQRSTUVWXZY"

allLetters :: String
allLetters = lowerLetters ++ upperLetters

main :: IO ()
main = do
  putStrLn "Please input a password:"
  password <- getLine
  if null password
  then main
  else do
    let (count, checkedPassword) = check1 (0,password)
    if count > 10
    then do
      putStrLn "Hit recursion limit of 10 password checks. Please try again."
      main
    else do
      putStrLn $ "Performed " ++ show count ++ " password checks."
      if password == checkedPassword
      then putStrLn "Password satisfies all criteria."
      else do
        putStrLn "Password does not satisfies all criteria."
        putStrLn $ "Suggested password modifications: " ++ checkedPassword

main

Our program is fairly simple because it passes around only a tuple of two basic data types. 

Passing larger data around and updating it can get even more tedious when programs get bigger. 

For this reason the **Writer monad** was created that allows you to pass data in the background between different Writer monads and also update it. 

## The Writer monad

### Definition of the Writer monad

The Haskell definition of the `Writer` type is defined in terms of the `WriterT` monad transformer:
```haskell
type Writer w = WriterT w Identity
```

It is contained in the **Control.Monad.Trans.Writer** module that comes with the **transformers** package. 

As said in the previous lesson we will learn about Monad transformers in lesson 25.

You can also define the Writer monad without using a monad transformer that works equally as the one above. 

Also in this lesson will chose this aproach. Let's first look at the definition of the **Writer** type:
```haskell
newtype Writer w a = Writer {runWriter :: (a,w)}
```

We see that the `Writer` data constructor holds a variable that can be accessed with the name `runWriter`.

The variable is a tuple of type `(a,w)`.

We can now create a Monad instance for `Writer w` and not just `Writer`.

This means the type of our writing variable will remain the same as we compose our function with `(>>=)`.
```haskell
(>>=) :: Writer w a        ->
         (a -> Writer w b) ->
         Writer w b
```

The monad instance for `Writer w` would be:
```haskell
instance (Monoid w) => Monad (Writer w) where
    return a             = Writer (a, mempty)
    (Writer (a,w)) >>= f = let (a',w') = runWriter $ f a 
                           in Writer (a', w `mappend` w')
```

We also need to define the Functor and Applicative instances.
```haskell
instance Functor (Writer w) where
fmap f (Writer (a,w)) = Writer (f a, w)

instance (Monoid w) => Applicative (Writer w) where
pure a = Writer (a, mempty)
(Writer (f,w)) <*> (Writer (a,w')) =  
    let b = f a
    in Writer (b, w `mappend` w')
```

### Helper functions

The **Control.Monad.Trans.Writer** module also defines some helper functions for the Writer moand. 

You have the option to import this module as a lazy or a strict module by appending **.Lazy** or **.Strict** to the module name.

The `tell` function appends data to the writer variable. It is used inside of the Writer monad.
```haskell
tell :: Monoid w => w -> Writer w ()
tell msg = Writer ((),msg)
```

The `writer` function does the oposite as the `runWriter` function. It takes a tuple and creates a Writer monad.
```haskell
writer :: Monoid w => (a,w) -> Writer w a
writer ~(a, w) = do
    tell w
    return a
```

The `listen` function retrieves the writer variable and the variable of type `a` from the Writer monad.
```haskell
listen :: Monoid w => Writer w a -> Writer w (a, w)
listen (Writer (a,w)) = Writer ((a,w),w)
```

It can be used inside or outside of a Writer Monad. To get both variable you also need to use the `<-` oprator that takes variables out of the monadic context.
```haskell
(a,w) <- listen writerMonad
```

The `pass` function takes in a Writer monad that holds a function which then gets applied to the writer variable and the updated Writer monad is returned. 
```haskell
pass :: Monoid w => Writer (a, w -> w) -> Writer w a
pass (Writer ((a,f),w)) = Writer (a,f w)
```

It can be used inside our outside or a Writer monad. 

The `listens` function takes in a function and a writer monad. Then it applies the function to the writer variable and returns the updated writer monad.
```haskell
listens :: Monoid w => (w -> b) -> Writer w a -> Writer w (a, b)
listens f writerMonad = do
    ~(a, w) <- listen writerMonad
    return (a, f w)
```

The tilde `~` sign in this expression means ... TODO

The `censor` function takes in a function and a writer monad and applies this function to the writer variable to update the writer monad.
```haskell
censor :: Monoid w => (w -> w) -> Writer w a -> Writer w a
censor f writerMonad = pass $ do
    a <- writerMonad
    return (a, f)
```

### MonadWriter type class

The Writer monad and the helper functions presented in the previous two chapters can be also found in the **Control.Monad.Writer** module.

This module is part of the **mtl** package that as said in the previous chapter is extending the **transformers** package.

The module defines the `MonadWriter` type class that contains the functions `writer`, `tell`, `listen` and `pass`.

They are equivalent to the functions describer in the previous chapter.

```haskell
class (Monoid w, Monad m) => MonadWriter w m | m -> w where
    writer :: (a,w) -> m a
    writer ~(a, w) = do
      tell w
      return a

    tell   :: w -> m ()
    tell w = writer ((),w)

    listen :: m a -> m (a, w)

    pass   :: m (a, w -> w) -> m a
    {-# MINIMAL (writer | tell), listen, pass #-}
```

The minimal complete definition are the `listen`, `pass` and either `writer` or `tell` functions. 

Same as for the `MonadReader` type classes you need to use also here the *MultiParamTypeClasses* and *FunctionalDependencies* language pragma.

For our Writer monad we could write an instance of the `MonadWriter` type class as:
```haskell
instance (Monoid w) => MonadWriter w (Writer w) where
    pass :: Monoid w => Writer w (a, w -> w) -> Writer w a
    pass (Writer ((a,f),w)) = Writer (a,f w)

    listen :: Monoid w => Writer w a -> Writer w (a, w)
    listen (Writer (a,w)) = Writer ((a,w),w)
    
    tell :: Monoid w => w -> Writer w ()
    tell msg = Writer ((),msg)
```

Also here you would need to include the language pragmas *FlexibleInstances* and *InstanceSigs*.

What all four language pragmas do was already explained in the Reader monad lesson. 

### Simple example

Here is a simple Writer monad example that uses the `tell` and `runWriter` functions.

In [ ]:
import Control.Monad.Writer

logAge :: Int -> Writer [String] Int  
logAge x = do
    tell ["Age is: " ++ show x]
    return x            
      
sumAge :: Writer [String] Int  
sumAge = do  
    a <- logAge 16  
    b <- logAge 18
    tell ["Summed age is: " ++ (show $ a + b)]
    return (a + b)

main :: IO ()
main = do
    let (age, messages) = runWriter sumAge
    mapM_ putStrLn messages

main

We see that every time we use the `tell` function the value gets appended to the first parameter of the Writer monad. 

The type of the parameter which is in our case a list has to have an instance of the **Monoid** type class, which lists have.

## Writer monad examples

Let's implement the example from the first chapter with the use of the Writer monad.

In [ ]:
instance Semigroup Int where
  a <> b = a + b

instance Monoid Int where
  mempty = 0

check1' :: String -> (Int, String)
check1' input = if length input < 5
  then swap $ runWriter $ do
    tell 1
    check2' $ input ++ "_below5"
  else swap $ runWriter $ do
    tell 1
    check3' $ input ++ "_above4"
  where swap (x,y) = (y,x)

check2' :: String -> Writer Int String
check2' input = if length input < 10
  then do
    return input
  else do
    tell 1
    return $ input ++ "_above9"

check3' :: String -> Writer Int String
check3' input = if length input < 16
  then do
    tell 1
    check2' $ removeFirst4 input
  else do
    tell 1
    check4' $ input ++ "_above15"
  where removeFirst4 xs = [(!! id) xs | id <- [3..((length xs) - 1)]]

check4' :: String -> Writer Int String
check4' input = if length input < 10
  then do
    tell 1
    return $ input ++ "_above29"
  else do
    tell 1
    return $ input ++ "_below30"

main :: IO ()
main = do
  let (count1, result1) = check1' "test"
  print "Processing word: test"
  print $ "Operations performed: " ++ show count1
  print $ "Final result: " ++ result1
  print "------------------------"
  let (count2, result2) = check1' "testing"
  print "Processing word: testing"
  print $ "Operations performed: " ++ show count2
  print $ "Final result: " ++ result2

main

Since the type of the write parameter has to have an instance of Monoid we implement it in the begining for `Int`. 

After that we define the check functions similar as in the previous example where we use the `tell` function to keep track of the operations we have performed. 

Here is another example where we use also the `listen` function that enables us to retrieve the logs from the Writer monad.

In [ ]:
type Adding a = Writer [String] a

logMsg :: String -> Adding ()
logMsg msg = tell [msg]

add1 :: Int -> Adding Int
add1 x = do
    logMsg "Starting add1."
    let y = x + 1
    logMsg $ "Computed result: " ++ show y
    return y

start :: Int -> Adding Int
start x = do
    (n, logs) <- listen $ add1 x
    let logLines = length logs
    logMsg $ "add1 logged " ++ show logLines ++ " lines"
    return $ n

main :: IO ()
main = do
    print "Input an integer number:"
    n <- (read <$> getLine) :: IO Int
    let (result, logs) = runWriter $ start n
    print $ "Result is: " ++ show result
    putStrLn "Logs are: "
    mapM_ print logs

main

In the final example we demonstrate how the `pass` function works. We use the code from the simple example chapter that we modify a bit.

In [ ]:
logAge :: Int -> Writer [String] Int  
logAge x = do
    tell ["Age is: " ++ show x]
    return x            
      
sumAge :: Writer [String] (Int, [String] -> [String])
sumAge = do  
    a <- logAge 16  
    b <- logAge 18
    tell ["Summed age is: " ++ (show $ a + b)]
    return (a + b,transform)

process :: Writer [String] Int
process = do
    pass sumAge

transform :: [String] -> [String]
transform list = map ("NOTE: " ++) list

main :: IO ()
main = do
    let (age, messages) = runWriter process
    mapM_ putStrLn messages
    
main

Note how we also had to change the type signature of the `sumAge` writer monad to include the `transform` functions type signature.

## Recap

In this lesson we've discussed:

- the motivation for introducing the Writer monad 

- the definition of the Writer monad type

- helper functions that work with the Writer monad

- examples that use the Writer monad type and its helper functions